In [ ]:
import qiskit as qt
import numpy as np
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit import Aer
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, AerPauliExpectation, CircuitStateFn, CircuitOp, MatrixExpectation
from qiskit_nature.circuit.library import HartreeFock
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter
import matplotlib.pyplot as plt


def expectation(operator, state):
    backend_i = Aer.get_backend('statevector_simulator')
    psi = CircuitStateFn(state)
    measurable_expression = StateFn(operator, is_measurement=True).compose(psi)
    expectation_i = PauliExpectation().convert(measurable_expression)
    sampler = CircuitSampler(backend_i).convert(expectation_i)
    electronic_energy = np.real(sampler.eval())
    return electronic_energy

z_coord = np.linspace(0.2,3.5,20)
energy = list()
for kk in z_coord:
  coord = 'H 0.0 0.0 0.0; H 0.0 0.0 '+str(kk)
  driver = PySCFDriver(atom=coord, charge=0, spin=0, basis='sto3g')
  es_problem = ElectronicStructureProblem(driver)

  # obtaining qubit Hamiltonian
  mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
  second_q_op = es_problem.second_q_ops()
  qubit_op = converter.convert(second_q_op['ElectronicEnergy'])

  # obtaining some properties
  # print(es_problem.grouped_property_transformed)

  es_particle_number = es_problem.grouped_property_transformed.get_property('ParticleNumber')
  num_particles = (es_particle_number.num_alpha, es_particle_number.num_beta)
  num_spin_orbitals = es_particle_number.num_spin_orbitals
  es_energy = es_problem.grouped_property_transformed.get_property('ElectronicEnergy')
  nuclear_repulsion_energy = es_energy.nuclear_repulsion_energy
#  print('Number Of Particles: ', num_particles)
#  print('Number of Spin Orbitals: ', num_spin_orbitals)
#  print('Nuclear Repulsion Energy: ', nuclear_repulsion_energy)

  # generating Hartree Fock state
  init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

  electronic_energy = expectation(qubit_op, init_state)
  total_energy = electronic_energy+nuclear_repulsion_energy
  energy.append(total_energy)

print(energy)
plt.plot(z_coord, energy)
plt.show()